# File Name: simple_prompt_mgmt.ipynb
### Location: Chapter 15
### Purpose: 
#####       1. Create a prompt using Prompt management
#####       2. Modify a prompt using Prompt management 
#####       3. Create a version of a prompt in Prompt management  
#####       4. Retrieve detail of prompt 
#####       5. Testing the prompt with simple way 
#####       6. Delete a version of a prompt in Prompt management
##### Dependency: Not Applicable
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"  

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import warnings
import time

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

## Define important environment variable

In [ ]:
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    # This session is responsible for managing credentials and region configuration
    boto3_session = boto3.session.Session()

    # Retrieve the current AWS region from the session (e.g., 'us-east-1', 'us-west-2')
    aws_region_name = boto3_session.region_name
    
    # Initialize Bedrock and Bedrock Runtime and Bedrock Agent clients using Boto3
    # These clients will allow interactions with Bedrock-related AWS services
    boto3_bedrock_client = boto3.client('bedrock', region_name=aws_region_name)
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=aws_region_name)
    boto3_bedrock_agent_client = boto3.client(service_name="bedrock-agent", region_name=aws_region_name)
    
    # Define the name of the prompt 
    prompt_name = "iPhone_accessories_recommendation"

    # Store all relevant variables in a dictionary for easier access and management
    variables_store = {
        "aws_region_name": aws_region_name,                          # AWS region name
        "boto3_bedrock_client": boto3_bedrock_client,                # Bedrock client instance
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client,  # Bedrock Runtime client instance
        "boto3_bedrock_agent_client": boto3_bedrock_agent_client,  # Bedrock agent client instance
        "prompt_name": prompt_name,
        "boto3_session": boto3_session                               # Current Boto3 session object
    }

    # Print all stored variables for debugging and verification
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

# Handle any exceptions that occur during the execution
except Exception as e:
    # Print the error message if an unexpected error occurs
    print(f"An unexpected error occurred: {e}")

# Create a prompt using Prompt management

#### 1. Purpose: To create a prompt with placeholders for dynamic inputs, allowing tailored text generation for a specific use case (e.g., recommending iPhone accessories).

#### 2. Prompt Details:
###### Name: "iPhone_accessories_recommendation"
###### Description: "Initial prompt"
###### Variants: Defines configurations for inference, such as the model, template type, and placeholders.

#### 3. Inference Configuration:
###### Uses the amazon.titan-text-express-v1 model.
###### Configures the output generation randomness using a temperature setting (0.8).

#### 4. Template Configuration:
###### A template string with placeholders:
###### "Recommend {{number_of_accessories}} popular accessories under {{dollar_amount}} for a {{customer_persona}} who recently bought an iPhone."

#### 5. Workflow:
###### Calls the create_prompt method using Boto3's Bedrock client to register the prompt and its variants.
###### Retrieves and prints the prompt_id from the API response for reference.

Refer: https://boto3.amazonaws.com/v1/documentation/api/1.35.6/reference/services/bedrock-agent/client/create_prompt.html

In [ ]:
try:
    # Create the prompt with different variants and configurations
    response = boto3_bedrock_agent_client.create_prompt(
        name=prompt_name,  # Name of the prompt
        description="Initial prompt",  # Description of the prompt
        variants=[  # Define variants of the prompt
            {
                "name": "Variant1",  # Name of the variant
                "modelId": "amazon.titan-text-express-v1",  # Model to be used for inference
                "templateType": "TEXT",  # Specify template type as TEXT
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.8  # Set temperature for randomness in the generated output
                    }
                },
                "templateConfiguration": {
                    "text": {
                        "text": "Recommend {{number_of_accessories}} popular accessories under {{dollar_amount}} for a {{customer_persona}} who recently bought an iPhone."  # Prompt template with placeholders
                    }
                }
            }
        ]
    )
    
    # Retrieve the prompt ID from the response
    prompt_id = response.get("id")
    print(f"Prompt created successfully with ID: {prompt_id}")

except boto3.exceptions.Boto3Error as e:
    # Handle any boto3 exceptions that may arise during API interaction
    print(f"An error occurred while creating the prompt: {str(e)}")
    
except Exception as e:
    # Catch any other unexpected errors
    print(f"An unexpected error occurred: {str(e)}")

# List all the prompts 

Refer: https://boto3.amazonaws.com/v1/documentation/api/1.35.6/reference/services/bedrock-agent/client/list_prompts.html#list-prompts

In [ ]:
# List prompts that you've created
boto3_bedrock_agent_client.list_prompts() 

# get detail information of a speciic prompt

Refer: https://boto3.amazonaws.com/v1/documentation/api/1.35.6/reference/services/bedrock-agent/client/get_prompt.html#get-prompt

In [ ]:
# Get information about the prompt that you created
response = boto3_bedrock_agent_client.get_prompt(promptIdentifier=prompt_id)

print(response)

# Create a version of a prompt in Prompt management

#### 1. Purpose: Create a new version of an existing prompt using AWS Bedrock's create_prompt_version method and provides error handling for any issues encountered.
###### Uses create_prompt_version via the Bedrock Agent client (boto3_bedrock_agent_client) to generate a new version of the prompt.

Refer: https://boto3.amazonaws.com/v1/documentation/api/1.35.6/reference/services/bedrock-agent/client/create_prompt_version.html#create-prompt-version

In [ ]:
try:
    # Attempt to create a new version of the prompt
    response = boto3_bedrock_agent_client.create_prompt_version(promptIdentifier=prompt_id)
    
    # Extract the version and ARN from the response
    prompt_version = response.get("version")  # Retrieve the newly created version number
    prompt_version_arn = response.get("arn")  # Retrieve the ARN of the newly created version

    # Print the response details for verification
    print("Prompt version created successfully!")
    print(f"Version: {prompt_version}")
    print(f"ARN: {prompt_version_arn}")

except Exception as e:
    # Handle any exceptions that occur during the request
    print(f"Error creating prompt version: {e}")

# List prompts that you've created
#### 1. Details of a specific version of a prompt using AWS Bedrock's get_prompt method and handles potential errors during the API call.

In [ ]:
try:
    # Retrieve the details of the specified prompt version
    response = boto3_bedrock_agent_client.get_prompt(
        promptIdentifier=prompt_id,
        promptVersion=prompt_version
    )

    # Print the response to view the prompt details
    print("Prompt details retrieved successfully!")
    print(response)

except Exception as e:
    # Handle any errors that occur during the API call
    print(f"Error retrieving prompt details: {e}")

# Modify a prompt using Prompt management

#### 1. Use cases: Updating Model ID and inferenceConfiguration

#### 2. Purpose: Updates an existing prompt by adding a new variant using the AWS Bedrock API. It attempts to modify the configuration of the prompt, specifically by adding a new variant with a different model and inference settings.
##### Calls update_prompt from the Bedrock Agent client (client):
##### name: Specifies the name of the prompt to be updated.
##### promptIdentifier: Specifies the unique identifier (ID) of the prompt.
##### variants: A list of new variants to add to the prompt. Each variant includes:
##### name: The name of the variant (e.g., "Variant2").
##### modelId: The model ID used for inference (e.g., "amazon.titan-text-premier-v1:0").
##### templateType: Specifies the type of template (e.g., TEXT).
##### inferenceConfiguration: Configurations for text generation (e.g., temperature, topP).
##### templateConfiguration: The template for the prompt, with placeholders to be filled dynamically during execution (e.g., for recommending accessories).

Refer: https://boto3.amazonaws.com/v1/documentation/api/1.35.6/reference/services/bedrock-agent/client/update_prompt.html#update-prompt

In [ ]:
try:
    # Update the prompt with a new variant
    response = boto3_bedrock_agent_client.update_prompt(
        name=prompt_name,  # The name of the prompt you're updating
        promptIdentifier=prompt_id,    # The unique identifier for the prompt
        variants=[  # Define variants of the prompt
            {
                "name": "Variant2",  # Name of the variant
                "modelId": "amazon.titan-text-premier-v1:0",  # Model used for inference
                "templateType": "TEXT",  # Define the template type as TEXT (for text generation)
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.8,  # Temperature controls the randomness of the response
                        "topP": 0.9  # Controls diversity via nucleus sampling
                    }
                },
                "templateConfiguration": {
                    "text": {
                        # Define the prompt template with placeholders for dynamic values
                        "text": "Recommend {{number_of_accessories}} popular accessories under {{dollar_amount}} for a {{customer_persona}} who recently bought an iPhone."
                    }
                }
            }
        ]
    )
    
    # Print the response to confirm the update
    print("Prompt updated successfully!")
    print(response)

except Exception as e:
    # Handle any errors during the update process
    print(f"Error updating prompt: {e}")

In [ ]:
# Create a version of a prompt in Prompt management
try:
    # Attempt to create a new version of the prompt
    response = boto3_bedrock_agent_client.create_prompt_version(promptIdentifier=prompt_id)
    
    # Extract the version and ARN from the response
    prompt_version = response.get("version")  # Retrieve the newly created version number
    prompt_version_arn = response.get("arn")  # Retrieve the ARN of the newly created version

    # Print the response details for verification
    print("Prompt version created successfully!")
    print(f"Version: {prompt_version}")
    print(f"ARN: {prompt_version_arn}")

except Exception as e:
    # Handle any exceptions that occur during the request
    print(f"Error creating prompt version: {e}")

In [ ]:
# List prompts that you've created
try:
    # Retrieve the details of the specified prompt version
    response = boto3_bedrock_agent_client.get_prompt(
        promptIdentifier=prompt_id,
        promptVersion=prompt_version
    )

    # Print the response to view the prompt details
    print("Prompt details retrieved successfully!")
    print(response)

except Exception as e:
    # Handle any errors that occur during the API call
    print(f"Error retrieving prompt details: {e}")

# Note: 
### Compare between version 1 and version 2. You will see change of Model Id and inferenceConfiguration

# Test a prompt using Prompt management

### Method 1. You can test the prompt through Amazon Bedrock Prompt Flow. You will learn in Chapter 16
### Method 2. You can test the prompt through Amazon Bedrock console. Refer https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management-test.html
### Method 3. Here, you will learn one simple way to retrive information about prompt and test. This is only to showcase the capability. You will always use Method 1 mentioned above. 



In [ ]:
# Common functions

# generate_dynamic_prompt
'''
The function generate_dynamic_prompt is designed to replace placeholders within a prompt template with specific values from a dictionary of parameters. 
This function allows for dynamic generation of prompts by filling in values for placeholders like {{number_of_accessories}}, {{dollar_amount}}, and {{customer_persona}}.
generate_dynamic_prompt(template, parameters): This function takes two arguments:
template: A string that contains placeholders (e.g., {{number_of_accessories}}).
parameters: A dictionary where the keys match the placeholders in the template, and the values are the ones that will replace those placeholders.
The function loops through the dictionary (parameters.items()), replacing each placeholder (e.g., {{number_of_accessories}}) in the template with its corresponding value (3 in the case of number_of_accessories).
'''

# Dynamic Parameters:
'''
The dynamic_parameters dictionary defines the values that will replace the placeholders in the prompt template.
"number_of_accessories": 3: This will replace {{number_of_accessories}} with 3.
"dollar_amount": 50: This will replace {{dollar_amount}} with 50.
"customer_persona": "budget-conscious college student": This will replace {{customer_persona}} with "budget-conscious college student".
'''

# Function to dynamically populate placeholders in the prompt
def generate_dynamic_prompt(template, parameters):
    """
    Replace placeholders in the template with actual values from parameters.
    :param template: The template string with placeholders (e.g., {{placeholder}})
    :param parameters: A dictionary of values to replace placeholders with
    :return: A populated string
    """
    for key, value in parameters.items():
        template = template.replace(f"{{{{{key}}}}}", str(value))
    return template

# Define dynamic input parameters for the prompt
dynamic_parameters = {
    "number_of_accessories": 3,
    "dollar_amount": 50,
    "customer_persona": "budget-conscious college student"
}

# Interact with the AWS Bedrock service to retrieve information about a specific prompt, generate a dynamic prompt based on a template, and invoke the model to generate a response. 
### 1. Get information about the prompt version
### 2. Extract information from the response
### 3. Retrieve or set default values for temperature and topP
### 4. Define dynamic parameters
### 5. Generate the dynamic prompt:
### 6. Prepare the input payload for model invocation
### 7. Invoke the model
### 8. Parse and display the model output

In [ ]:
try:
    # Get information about the specified prompt version
    response = boto3_bedrock_agent_client.get_prompt(
        promptIdentifier=prompt_id,
        promptVersion=prompt_version
    )
    
    # Extract relevant information from the response
    prompt_arn = response.get("arn")
    model_id = response['variants'][0]['modelId']
    template = response['variants'][0]['templateConfiguration']['text']['text']
    
    # Print details for verification
    print(f"Prompt ARN: {prompt_arn}")
    print(f"Model ID: {model_id}")
    print(f"Template: {template}")
    
    # Extract the inference configuration settings (temperature, topP)
    inferenceConfiguration_outcome = response['variants'][0]['inferenceConfiguration']['text']
    temperature = inferenceConfiguration_outcome.get('temperature', 0.5)  # Default value if 'temperature' is not present
    topP = inferenceConfiguration_outcome.get('topP', 0.5)  # Default value if 'topP' is not present
    
    # Define dynamic parameters for the prompt (example: number_of_accessories, dollar_amount)
    dynamic_parameters = {
        "number_of_accessories": 5,
        "dollar_amount": 100,
        "customer_persona": "tech-savvy"
    }

    # Generate the dynamic prompt based on the template and dynamic parameters
    dynamic_prompt = generate_dynamic_prompt(template, dynamic_parameters)
    print(f"Generated Prompt: {dynamic_prompt}\n")

    # Prepare the input payload for the model invocation
    body = json.dumps({
        "inputText": dynamic_prompt,
        "textGenerationConfig": {
            "topP": topP,          # Controls the nucleus sampling probability (diversity of output)
            "temperature": temperature  # Controls the creativity of the model's response
        }
    })
    
    modelId = model_id  # Model ID to be used in the model invocation
    accept = "application/json"
    contentType = "application/json"

    # Invoke the model with the specified parameters using the Bedrock runtime client
    response = boto3_bedrock_runtime_client.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )
    
    # Parse the response body to extract the model's output
    response_body = json.loads(response.get("body").read())
    output_text = response_body.get("results", [{}])[0].get("outputText", "No output text available")

    # Display the output text from the model
    print("Model Output:", output_text)

except boto3.exceptions.Boto3Error as boto3_err:
    # Handle AWS-specific errors
    print(f"An AWS error occurred: {str(boto3_err)}")

except Exception as err:
    # Handle unexpected errors
    print(f"An unexpected error occurred: {str(err)}")


# Delete a version of a prompt in Prompt management

### Delete a specific version of a prompt in Prompt management

In [ ]:
try:
    # Delete the specified prompt version
    response = boto3_bedrock_agent_client.delete_prompt(
        promptIdentifier=prompt_id,
        promptVersion=prompt_version
    )
    
    # Print the response to confirm the deletion
    print("Prompt version deleted successfully!")
    print(response)

except boto3.exceptions.Boto3Error as boto3_err:
    # Handle AWS-specific errors
    print(f"An AWS error occurred while deleting the prompt: {str(boto3_err)}")

except Exception as err:
    # Handle unexpected errors
    print(f"An unexpected error occurred: {str(err)}")

### Delete a prompt in Prompt management

In [ ]:
try:
    # Delete the specified prompt
    response = boto3_bedrock_agent_client.delete_prompt(
        promptIdentifier=prompt_id
    )
    
    # Print the response to confirm the deletion
    print("Prompt deleted successfully!")
    print(response)

except boto3.exceptions.Boto3Error as boto3_err:
    # Handle AWS-specific errors
    print(f"An AWS error occurred while deleting the prompt: {str(boto3_err)}")

except Exception as err:
    # Handle unexpected errors
    print(f"An unexpected error occurred: {str(err)}")

# End of NoteBook 

## Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

## Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html